<a href="https://colab.research.google.com/github/suryasuresh91/Lexical-Complexity-Prediction/blob/main/LCP_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-m7wntiq8
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-m7wntiq8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 40.9 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3178120 sha256=b351f1419c84651693c546f81c766b4ed87e8f259ae9edf1dd59d3cbc30ad106
  Stored in directory: /tmp/pip-ephem-wheel-cache-gh_4ulw9/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfu

In [3]:
import numpy as np
import pandas as pd
import torch
import csv
from scipy import stats
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

In [4]:
torch.manual_seed(0)

In [5]:
SINGLE_TRAIN_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv"
SINGLE_TEST_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_single_test.tsv"
single_train_filepath = "/content/sample_data/single_train.csv"
single_test_filepath = "/content/sample_data/single_test.csv"

In [6]:
k = 1            # token append number

In [7]:
def prepare_dataset(TRAIN_DATAPATH, TEST_DATAPATH, train_filepath, test_filepath):
    df_train = pd.read_csv(TRAIN_DATAPATH, sep = '\t', quotechar="'", quoting = csv.QUOTE_NONE)
    df_test = pd.read_csv(TEST_DATAPATH, sep = '\t', quotechar="'", quoting = csv.QUOTE_NONE)
    df_train['complexity'] = df_train['complexity'].astype(float)
    df_test['complexity'] = df_test['complexity'].astype(float)
    for i in range(len(df_train)):
        first = str(df_train['token'][i]) + " [SEP] "
        last = " [SEP] " + str(df_train['token'][i])
        for _ in range(k):
            df_train['sentence'][i] = first + df_train['sentence'][i] + last
    for i in range(len(df_test)):
        first = str(df_test['token'][i]) + " [SEP] "
        last = " [SEP] " + str(df_test['token'][i])
        for _ in range(k):
            df_test['sentence'][i] = first + df_test['sentence'][i] + last
    df_train = df_train.drop(['id', 'corpus', 'token'], axis = 1)
    df_test = df_test.drop(['id', 'corpus', 'token'], axis = 1)
    df_train = df_train[['complexity', 'sentence']]
    df_test = df_test[['complexity', 'sentence']]
    df_train.to_csv(train_filepath, index = False)
    df_test.to_csv(test_filepath, index = False)
   

In [8]:
device = 'cuda'
batch_size = 4
num_epochs = 8

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

In [10]:
def prepare_iterators(train_filepath, test_filepath):
    label = Field(sequential = False, use_vocab = False, batch_first = True, dtype = torch.float32)
    text = Field(use_vocab = False, tokenize = tokenizer.encode, lower = False, batch_first = True, pad_token = PAD_INDEX, unk_token = UNK_INDEX)
    fields = [('complexity', label), ('sentence', text)]
    train = TabularDataset(path = train_filepath, format = 'csv', skip_header = True, fields = fields)
    train_iter = BucketIterator(train, batch_size = batch_size, sort_key = lambda x: len(x.sentence), device = device, train = True, sort = True, sort_within_batch = True)
    test_label = Field(sequential = False, use_vocab = False, batch_first = True, dtype = torch.float32)
    test_text = Field(use_vocab = False, tokenize = tokenizer.encode, lower = False, batch_first = True, pad_token = PAD_INDEX, unk_token = UNK_INDEX)
    test_fields = [('complexity', test_label), ('sentence', test_text)]
    test = TabularDataset(path = test_filepath, format = 'csv', skip_header = True, fields = test_fields)
    test_iter = BucketIterator(test, batch_size = batch_size, sort_key = lambda x: len(x.sentence), device = device, train = False, sort = True, sort_within_batch = True)
    return train_iter, test_iter

In [11]:
model = BertForSequenceClassification.from_pretrained("bert-large-uncased")
model.config.num_labels = 1
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.00001)

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [12]:
def train_model(model, iterator):
    epoch_loss = 0.0
    model.train()
    for batch in iterator:
        text = batch.sentence
        label = batch.complexity
        optimizer.zero_grad()
        output = model(text)                       
        logits = output.logits[:, : 1]                    
        logits = torch.sigmoid(torch.squeeze(logits))
        try:
            predicted.extend(logits.tolist())
            labels.extend(label.tolist())
            loss = criterion(label, logits)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        except TypeError:
            pass
    return epoch_loss / len(iterator)


In [13]:
def test_model(model, iterator):
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.sentence
            label = batch.complexity
            output = model(text)
            logits = output.logits[:, : 1]                    
            logits = torch.sigmoid(torch.squeeze(logits))
            try:
                test_predicted.extend(logits.tolist())
                test_labels.extend(label.tolist())
            except TypeError:
                pass

In [14]:
def calculate_metrics(y, y_hat):
    vx = y.astype(float)
    vy = y_hat.astype(float)
    pearsonR = np.corrcoef(vx, vy)[0, 1]
    spearmanRho = stats.spearmanr(vx, vy)
    MSE = np.mean((vx - vy) ** 2)
    MAE = np.mean(np.absolute(vx - vy))
    RSquared = (pearsonR ** 2)

    print("Pearson's R: {}".format(pearsonR))
    print("Spearman's rho: {}".format(spearmanRho))
    print("R Squared: {}".format(RSquared))
    print("MSE: {}".format(MSE))
    print("MAE: {}".format(MAE))

In [15]:
prepare_dataset(SINGLE_TRAIN_DATAPATH, SINGLE_TEST_DATAPATH, single_train_filepath, single_test_filepath)
train_iter, test_iter = prepare_iterators(single_train_filepath, single_test_filepath)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
print("++++++Running for single+++++")
for epoch in range(num_epochs):
    labels = []
    predicted = []
    train_loss = train_model(model, train_iter)
    print(f'\t Epoch: {epoch + 1} | Train Loss: {train_loss: }')
    print("------Metrics for train------")
    calculate_metrics(np.array(labels), np.array(predicted))
    test_labels = []
    test_predicted = []
    test_model(model, test_iter)
    print("------Metrics for test-------")
    calculate_metrics(np.array(test_labels), np.array(test_predicted))

++++++Running for single+++++
	 Epoch: 1 | Train Loss:  0.011295770972182803
------Metrics for train------
Pearson's R: 0.6147618789333266
Spearman's rho: SpearmanrResult(correlation=0.5268217410564554, pvalue=0.0)
R Squared: 0.37793216778963407
MSE: 0.011297762791066432
MAE: 0.08168695127692135
------Metrics for test-------
Pearson's R: 0.7418303689326251
Spearman's rho: SpearmanrResult(correlation=0.7051492634144232, pvalue=1.248015097610422e-138)
R Squared: 0.5503122962707147
MSE: 0.008214487059711067
MAE: 0.0700695018597175
